In [67]:
# Добавляем # slots:, если его нет. Работает, если есть # intent:
def process_conll_file(input_filename, output_filename):
    with open(input_filename, "r") as f:
        read = f.read()
    lines = read.split('\n')
    output_lines = []
    found_intent = False  # Флаг для отслеживания строки intent

    # Проходим по каждой строке
    for i, line in enumerate(lines):
        # Добавляем текущую строку в список выходных строк
        output_lines.append(line)

        # Проверяем, начинается ли строка с "# intent:"
        if line.startswith('# intent:'):
            # Проверяем наличие строки # slots: после intent
            if (i + 1) < len(lines) and not lines[i + 1].startswith('# slots:'):
                # Добавляем строку # slots:
                output_lines.append('# slots:')

        # Если строка "# slots:" уже встретилась, сбрасываем флаг
        if line.startswith('# slots:'):
            found_intent = False

    # Объединяем список выходных строк в один текст
    output_text = '\n'.join(output_lines)
    
    # Записываем измененный текст в выходной файл
    with open(output_filename, "a") as f:
        f.write(output_text)

# Добавьте свой файл
process_conll_file('tat.test_id_50.4.conll', 'output.conll')


In [44]:
# Открываем файл для чтения и последующей записи
with open('ru.valid.conll', 'r+') as f:
    # Читаем содержимое файла
    read = f.read()
    # Разделяем содержимое файла на абзацы по пустой строке
    paragraphs = read.strip().split('\n\n')

    # Список для хранения обновленных абзацев
    updated_paragraphs = []

    # Проходим по каждому абзацу
    for paragraph in paragraphs:
        # Инициализируем список для хранения слотов
        intents = []
        # Разделяем абзац на строки
        lines = paragraph.strip().split('\n')
        # Получаем текст RU из абзаца
        ru_line = [line for line in lines if line.startswith('# text RU:')][0]
        ru_text = ru_line.split(':', 1)[1].strip()  # Извлекаем текст после '# text RU:'


        # Проходим по строкам абзаца
        for line in lines:
            # Игнорируем строки, которые не содержат данные о словах
            if line.startswith('#'):
                continue

            # Разделяем строку на элементы: индекс, слово, intent, mark
            parts = line.split('\t')
            if len(parts) != 4:
                continue

            index, word, intent, mark = parts
            intent = intent.strip()
            intents.append(intent)
        # print(intents[0])

        # Формируем строку для поля # intent
        intents_str = ''.join(intents[0])

        # Проверяем, есть ли уже строка # intent в абзаце
        found_intents_line = False
        updated_lines = []
        for line in lines:
            if line.startswith('# intent:'):
                # Если нашли строку # intent:, заменяем её на новую строку intents_str
                updated_lines.append(f'# intent: {intents_str}')
                found_intents_line = True
            else:
                updated_lines.append(line)

        # Добавляем обновленный абзац в список
        updated_paragraphs.append('\n'.join(updated_lines))

    # Переходим в начало файла и записываем обновленные абзацы
    f.seek(0)
    f.write('\n\n'.join(updated_paragraphs))
    f.truncate()  # Удаляем остаток содержимого файла, если оно есть

In [69]:
# Открываем файл для чтения и последующей записи
with open('tat.test_id_50.4.conll', 'r+') as f:
    # Читаем содержимое файла
    read = f.read()
    # Разделяем содержимое файла на абзацы по пустой строке
    paragraphs = read.strip().split('\n\n')

    # Список для хранения обновленных абзацев
    updated_paragraphs = []

    # Проходим по каждому абзацу
    for paragraph in paragraphs:
        # Инициализируем список для хранения слотов
        slots = []
        # Разделяем абзац на строки
        lines = paragraph.strip().split('\n')
        # Получаем текст TAT из абзаца
        tat_line = [line for line in lines if line.startswith('# text TAT:')][0]
        tat_text = tat_line.split(':', 1)[1].strip()  # Извлекаем текст после '# text TAT:'

        # Начальные значения для отслеживания текущего слота
        current_slot_start = None
        current_slot_end = None
        current_mark = None

        # Проходим по строкам абзаца
        for line in lines:
            # Игнорируем строки, которые не содержат данные о словах
            if line.startswith('#'):
                continue

            # Разделяем строку на элементы: индекс, слово, intent, mark
            parts = line.split('\t')
            if len(parts) != 4:
                continue

            index, word, intent, mark = parts
            mark = mark.strip()

            # Проверяем метку слова
            if mark.startswith('O'):
                # Если встретили 'O' и у нас есть текущий слот, завершаем его
                if current_slot_start is not None and current_slot_end is not None:
                    slots.append(f'{current_slot_start}:{current_slot_end}:{current_mark}')
                    current_slot_start = None
                    current_slot_end = None
                    current_mark = None
            else:
                # Извлекаем тип сущности из метки (удаляем префикс 'B-' или 'I-')
                entity_type = mark[2:]

                # Найдем начальный индекс слова в тексте TAT
                word_index = tat_text.find(word)

                if mark.startswith('B-'):
                    # Если слово начинается с 'B-', начинаем новый слот
                    if current_slot_start is not None and current_slot_end is not None:
                        slots.append(f'{current_slot_start}:{current_slot_end}:{current_mark}')
                    current_slot_start = word_index + 1  # Увеличиваем индекс на 1 для начала
                    current_slot_end = word_index + len(word)
                    current_mark = entity_type
                elif mark.startswith('I-') and current_mark == entity_type:
                    # Если слово начинается с 'I-' и совпадает с текущей меткой, продолжаем текущий слот
                    current_slot_end = word_index + len(word)

        # Если у нас есть незавершенный слот после прохождения абзаца, добавляем его
        if current_slot_start is not None and current_slot_end is not None:
            slots.append(f'{current_slot_start}:{current_slot_end}:{current_mark}')

        # Формируем строку для поля # slots
        slots_str = ', '.join(slots)

        # Проверяем, есть ли уже строка # slots в абзаце
        found_slots_line = False
        updated_lines = []
        for line in lines:
            if line.startswith('# slots:'):
                # Если нашли строку # slots:, заменяем её на новую строку slots_str
                updated_lines.append(f'# slots: {slots_str}')
                found_slots_line = True
            else:
                updated_lines.append(line)

        # Если строки # slots не было, добавляем её в конец абзаца
        if not found_slots_line:
            updated_lines.append(f'# slots: {slots_str}')

        # Добавляем обновленный абзац в список
        updated_paragraphs.append('\n'.join(updated_lines))

    # Переходим в начало файла и записываем обновленные абзацы
    f.seek(0)
    f.write('\n\n'.join(updated_paragraphs))
    f.truncate()  # Удаляем остаток содержимого файла, если оно есть

In [68]:
# Открываем файл для чтения и последующей записи
with open('tat.test_id_50.4.conll', 'r+', encoding='utf-8') as f:
    # Читаем содержимое файла
    content = f.read()
    # Разделяем содержимое файла на абзацы по пустой строке
    paragraphs = content.strip().split('\n\n')

    # Список для хранения обновленных абзацев
    updated_paragraphs = []

    # Проходим по каждому абзацу
    for paragraph in paragraphs:
        # Разделяем абзац на строки
        lines = paragraph.strip().split('\n')
        # Найдем индекс строки # slots:
        slots_line_index = -1
        for i, line in enumerate(lines):
            if line.startswith('# slots:'):
                slots_line_index = i
                break
        
        # Если нашли строку # slots:
        if slots_line_index != -1:
            # Начинаем проверку строк после строки # slots:
            for j in range(slots_line_index + 1, len(lines)):
                line = lines[j]
                # Разделяем строку по пробелам
                elements = line.split()
                # Если строка была разделена, то есть пробелы
                if len(elements) > 1:
                    # Объединяем элементы с табуляцией
                    tabbed_line = '\t'.join(elements)
                    # Обновляем строку с табуляцией
                    lines[j] = tabbed_line
        
        # Добавляем обновленный абзац в список
        updated_paragraphs.append('\n'.join(lines))

    # Переходим в начало файла и записываем обновленные абзацы
    f.seek(0)
    f.write('\n\n'.join(updated_paragraphs))
    f.truncate()  # Обрезаем остаток содержимого файла, если он есть

In [48]:
# Открываем файл для чтения
with open('tat.test_id_50.2.conll', 'r', encoding='utf-8') as f:
    read = f.read()
    # Разделяем содержимое файла на абзацы по пустой строке
    paragraphs = read.strip().split('\n\n')

# Проходим по каждому абзацу
for paragraph in paragraphs:
    # Инициализируем список для хранения слотов
    slots = []
    # Разделяем абзац на строки
    lines = paragraph.strip().split('\n')
    # Получаем текст TAT из абзаца
    tat_line = [line for line in lines if line.startswith('# text TAT:')][0]
    tat_text = tat_line.split(':', 1)[1].strip()  # Извлекаем текст после '# text TAT:'

    # Начальные значения для отслеживания текущего слота
    current_slot_start = None
    current_slot_end = None
    current_mark = None

    # Проходим по строкам абзаца
    for line in lines:
        # Игнорируем строки, которые не содержат данные о словах
        if line.startswith('#'):
            continue

        # Разделяем строку на элементы: индекс, слово, intent, mark
        parts = line.split('\t')
        if len(parts) != 4:
            continue

        index, word, intent, mark = parts
        mark = mark.strip()

        # Проверяем метку слова
        if mark.startswith('O'):
            # Если встретили 'O' и у нас есть текущий слот, завершаем его
            if current_slot_start is not None and current_slot_end is not None:
                slots.append(f'{current_slot_start}:{current_slot_end}:{current_mark}')
                current_slot_start = None
                current_slot_end = None
                current_mark = None
        else:
            # Извлекаем тип сущности из метки (удаляем префикс 'B-' или 'I-')
            entity_type = mark[2:]

            # Найдем начальный индекс слова в тексте TAT
            word_index = tat_text.find(word)

            if mark.startswith('B-'):
                # Если слово начинается с 'B-', начинаем новый слот
                if current_slot_start is not None and current_slot_end is not None:
                    slots.append(f'{current_slot_start}:{current_slot_end}:{current_mark}')
                current_slot_start = word_index + 1  # Увеличиваем индекс на 1 для начала
                current_slot_end = word_index + len(word)
                current_mark = entity_type
            elif mark.startswith('I-') and current_mark == entity_type:
                # Если слово начинается с 'I-' и совпадает с текущей меткой, продолжаем текущий слот
                current_slot_end = word_index + len(word)
    
    # Если у нас есть незавершенный слот после прохождения абзаца, добавляем его
    if current_slot_start is not None and current_slot_end is not None:
        slots.append(f'{current_slot_start}:{current_slot_end}:{current_mark}')

    # Формируем строку для поля # slots
    slots_str = ', '.join(slots)

    # Выводим поле # slots
    print(f'# slots: {slots_str}')


# slots: 14:32:object_name, 46:53:object_type
# slots: 5:10:playlist
# slots: 1:9:location, 11:18:party_size_number
# slots: 4:8:music_item, 10:18:reference, 20:29:playlist
# slots: 1:10:service, 12:25:artist, 27:32:sort, 34:38:music_item
# slots: 1:11:location
# slots: 1:5:location, 7:11:datetime, 13:17:condition_description
# slots: 5:8:datetime, 15:22:music_item
# slots: 6:24:location, 26:51:movie_name, 58:62:object_type
# slots: 1:7:object_select, 9:18:object_type, 20:24:best_rating, 26:26:rating_value
# slots: 1:13:artist, 15:19:music_item, 21:32:playlist
# slots: 5:5:party_size_number, 14:23:datetime, 25:29:location, 31:38:location, 40:44:cuisine, 46:54:restaurant_type
# slots: 5:16:object_name, 18:27:object_type
# slots: 1:2:object_select, 4:12:object_part_of_series_type, 14:14:rating_value
# slots: 1:21:object_name, 23:27:best_rating, 29:29:rating_value, 31:36:rating_unit
# slots: 1:5:reference, 7:18:playlist
# slots: 5:14:location, 16:19:cuisine, 21:30:restaurant_type
# slots:

In [243]:
# Открываем файл
with open ('en.valid_id_ann_alsu.conll', "r") as f:
    read = f.read()
    # Делим по абзацам
    lines = read.split('\n\n')

for i in lines:
    slots = []  # Тут будем хранить идексы слов для слотов
    line = i.split('\n')  # Делим по строкам
    for j in range(len(line)):
        text = line[j].split('\t')  # Делим по табуляции
        if len(text) == 4:  # Если количество элементов равно 4
            if not text[3].startswith('O'):  # Если элемент не ничинается с O
                if text[1] in line[3][12:]:  # Если слово есть в предложении после # text TAT:
                    s1 = int(line[3][12:].index(text[1]) + 1)  # Считаем начало слова
                    s2 = int(line[3][12:].index(text[1]) + len(text[1]))  # Считаем конец слова
                    mark = text[3][2:]  # Считываем название сущности
                    slots.append(f'{s1}:{s2}:{mark}')  # Приводим в нужный вид, как должно быть записано в # slots:
    print(slots)


['1:5:reference']
['1:5:reference', '7:11:reminder/todo', '13:15:reminder/todo']
['13:19:datetime', '21:24:datetime']
[]
[]
[]
['1:5:datetime', '7:14:datetime', '16:21:datetime', '23:26:datetime', '28:32:reminder/todo', '34:45:reminder/todo']
[]
['1:6:datetime', '8:11:weather/attribute']
['1:5:datetime', '7:12:location', '14:21:weather/attribute']
['13:17:datetime', '19:23:datetime', '25:25:datetime']
['1:2:datetime', '4:5:datetime', '7:16:datetime', '18:22:weather/attribute']
['1:5:datetime', '7:11:weather/attribute']
['13:19:datetime', '21:25:datetime', '27:31:datetime', '33:38:datetime', '40:43:datetime']
[]
['1:5:datetime', '7:11:datetime', '13:17:weather/attribute']
['13:19:datetime', '21:21:datetime']
['1:5:datetime', '12:16:weather/attribute']
[]
['33:41:weather/attribute']
['1:7:reminder/todo', '9:14:reminder/todo']
['1:5:datetime', '12:16:weather/attribute']
[]
['1:5:reference']
['1:10:datetime', '12:16:datetime', '18:23:datetime', '25:29:datetime']
['1:6:datetime', '8:14:date

In [15]:
with open ('en.valid_id_ann_alsu.conll', "r") as f:
    read = f.read()
    lines = read.split('\n\n')
for i in lines:
    lis = []
    line = i.split('\n')
    for j in range(len(line)):
        text = line[j].split('\t')
        print(text)
        if len(text) == 4:
            if not text[3].startswith('O'):
                if text[1] in line[3][12:]:
                    s1 = line[3][12:].index(text[1]) + 1
                    s2 = line[3][12:].index(text[1]) + len(text[1])
                    mark = text[3][2:]
                    slots = f'{s1}:{s2}:{mark}'
                    # print(f'{s1}:{s2}:{mark}')
                    lis.append

['# id: valid_101']
['# annotator: Alsu']
['# text EN: What are my reminders?']
['# text TAT: Минем нинди искә төшерүләрем бар?']
['# intent: reminder/show_reminders']
['# slots: ']
['1', 'Минем', 'reminder/show_reminders', 'B-reference']
['2', 'нинди', 'reminder/show_reminders', 'O']
['3', 'искә', 'reminder/show_reminders', 'O']
['4', 'төшерүләрем', 'reminder/show_reminders', 'O']
['5', 'бар', 'reminder/show_reminders', 'O']
['6', '?', 'reminder/show_reminders', 'O']
['# id: valid_102']
['# annotator: Alsu']
['# text EN: Cancel my reminder to text Dad']
['# text TAT: Минем әтиге язу турында хәтерләтүемне бетер']
['# intent: reminder/cancel_reminder']
['# slots: ']
['1', 'Минем', 'reminder/cancel_reminder', 'B-reference']
['2', 'әтиге', 'reminder/cancel_reminder', 'B-reminder/todo']
['3', 'язу', 'reminder/cancel_reminder', 'I-reminder/todo']
['4', 'турында', 'reminder/cancel_reminder', 'O']
['5', 'хәтерләтүемне', 'reminder/cancel_reminder', 'O']
['6', 'бетер', 'reminder/cancel_reminder

In [204]:
# Открываем файл
with open ('en.valid_id_ann_alsu.conll', "r") as f:
    read = f.read()
    # Делаем список по абзацам
    lines = read.split('\n\n')
lis = []
for i in lines:
    line = i.split('\n')
    print(line)
    for j in range(len(line)):
        text = line[j].split('\t')
        # print(text)
        if len(text) == 4:
            if not text[3].startswith('O'):
                if text[1] in line[3][12:]:
                    s1 = line[3][12:].index(text[1]) + 1
                    s2 = line[3][12:].index(text[1]) + len(text[1])
                    mark = text[3][2:]
                    slots = f'{s1}:{s2}:{mark}'
                    print(f'{s1}:{s2}:{mark}')
                    lis.append(slots)
                    
                    
# print(lis)

['# id: valid_101', '# annotator: Alsu', '# text EN: What are my reminders?', '# text TAT: Минем нинди искә төшерүләрем бар?', '# intent: reminder/show_reminders', '# slots: ', '1\tМинем\treminder/show_reminders\tB-reference', '2\tнинди\treminder/show_reminders\tO', '3\tискә\treminder/show_reminders\tO', '4\tтөшерүләрем\treminder/show_reminders\tO', '5\tбар\treminder/show_reminders\tO', '6\t?\treminder/show_reminders\tO']
1:5:reference
['# id: valid_102', '# annotator: Alsu', '# text EN: Cancel my reminder to text Dad', '# text TAT: Минем әтиге язу турында хәтерләтүемне бетер', '# intent: reminder/cancel_reminder', '# slots: ', '1\tМинем\treminder/cancel_reminder\tB-reference', '2\tәтиге\treminder/cancel_reminder\tB-reminder/todo', '3\tязу\treminder/cancel_reminder\tI-reminder/todo', '4\tтурында\treminder/cancel_reminder\tO', '5\tхәтерләтүемне\treminder/cancel_reminder\tO', '6\tбетер\treminder/cancel_reminder\tO']
1:5:reference
7:11:reminder/todo
13:15:reminder/todo
['# id: valid_103',

In [30]:
# Открываем файл
with open ('ru.valid.conll', "r") as f:
    read = f.read()
    # Делим по абзацам
    lines = read.split('\n\n')

for i in lines:
    slots = []  # Тут будем хранить идексы слов для слотов
    line = i.split('\n')  # Делим по строкам
    for j in range(len(line)):
        text = line[j].split('\t')  # Делим по табуляции
        print(text)
        if len(text) == 4:  # Если количество элементов равно 4
            print(text[2])  # Если элемент не ничинается с O
    #             if text[1] in line[3][12:]:  # Если слово есть в предложении после # text TAT:
    #                 s1 = int(line[3][12:].index(text[1]) + 1)  # Считаем начало слова
    #                 s2 = int(line[3][12:].index(text[1]) + len(text[1]))  # Считаем конец слова
    #                 mark = text[3][2:]  # Считываем название сущности
    #                 slots.append(f'{s1}:{s2}:{mark}')  # Приводим в нужный вид, как должно быть записано в # slots:
    # print(slots)

[' # text EN: Is it going to rain today?']
['# text RU: Будет ли дождь сегодня?']
['# slots: 10:14:weather/attribute, 16:22:datetime']
['1', 'Будет', 'weather/find', 'O']
weather/find
['2', 'ли', 'weather/find', 'O']
weather/find
['3', 'дождь', 'weather/find', 'B-weather/attribute']
weather/find
['4', 'сегодня', 'weather/find', 'B-datetime']
weather/find
['5', '?', 'weather/find', 'O']
weather/find
['# text EN: how hot is it going to be today']
['# text RU: насколько жарко будет сегодня']
['# slots: 11:15:weather/attribute, 23:29:datetime']
['1', 'насколько', 'weather/find', 'O']
weather/find
['2', 'жарко', 'weather/find', 'B-weather/attribute']
weather/find
['3', 'будет', 'weather/find', 'O']
weather/find
['4', 'сегодня', 'weather/find', 'B-datetime']
weather/find
['# text EN: How hot is it?']
['# text RU: Насколько жарко сейчас?']
['# slots: 11:15:weather/attribute']
['1', 'Насколько', 'weather/find', 'O']
weather/find
['2', 'жарко', 'weather/find', 'B-weather/attribute']
weather/fin

In [131]:
def update_slots(file_path):
    with open(file_path, 'r') as f:
        lines = f.read().strip().split('\n')

    blocks = []
    current_block = []

    # Разделение строк на блоки
    for line in lines:
        if line.strip() == '':
            if current_block:
                blocks.append(current_block)
                current_block = []
        else:
            current_block.append(line)

    # Если остался последний блок, добавляем его
    if current_block:
        blocks.append(current_block)

    # Обработка каждого блока
    for block in blocks:
        # Извлечение строки ID и текста
        id_line = block[0]
        text_tat_line = block[3]
        intent_line = block[4]

        # Извлечение текста на татарском
        text_tat = text_tat_line.split(': ')[1]

        # Извлечение намерения
        intent = intent_line.split(': ')[1]

        # Поиск строки # slots:
        slots_index = -1
        for i, line in enumerate(block):
            if line.startswith("# slots:"):
                slots_index = i
                break

        if slots_index == -1:
            continue  # Пропускаем блок, если нет строки # slots:

        # Обработка строк слотов
        updated_slots = []
        for slot_line in block[6:]:
            parts = slot_line.split('\t')
            if len(parts) == 4:
                word = parts[0]
                slot_label = parts[3]

                # Проверка, начинается ли метка с 'B-' или 'I-'
                if slot_label.startswith('B-') or slot_label.startswith('I-'):
                    # Находим начальный и конечный индекс слова в тексте на татарском
                    s1 = text_tat.find(word)
                    if s1 != -1:
                        s2 = s1 + len(word)
                        # Форматируем строку слота
                        updated_slots.append(f'{s1}:{s2}:{intent}')

        # Обновляем строку # slots: в блоке
        block[slots_index] = f'# slots: {", ".join(updated_slots)}'

    # Запись обновленного контента обратно в файл
    with open(file_path, 'w') as f:
        for block in blocks:
            for line in block:
                f.write(line + '\n')
            f.write('\n')

# Обновление слотов в файле
update_slots('en.valid_id_ann_alsu.conll')


In [22]:
# Открываем файл для чтения и последующей записи
with open('ru.valid.conll', 'r+') as f:
    # Читаем содержимое файла
    read = f.read()
    # Разделяем содержимое файла на абзацы по пустой строке
    paragraphs = read.strip().split('\n\n')

    # Список для хранения обновленных абзацев
    updated_paragraphs = []

    # Проходим по каждому абзацу
    for paragraph in paragraphs:
        # Инициализируем список для хранения слотов
        slots = []
        # Разделяем абзац на строки
        lines = paragraph.strip().split('\n')
        # Получаем текст TAT из абзаца
        tat_line = [line for line in lines if line.startswith('# text RU:')][0]
        tat_text = tat_line.split(':', 1)[1].strip()  # Извлекаем текст после '# text RU:'

        # Начальные значения для отслеживания текущего слота
        current_slot_start = None
        current_slot_end = None
        current_mark = None

        # Проходим по строкам абзаца
        for line in lines:
            # Игнорируем строки, которые не содержат данные о словах
            if line.startswith('#'):
                continue

            # Разделяем строку на элементы: индекс, слово, intent, mark
            parts = line.split('\t')
            if len(parts) != 4:
                continue

            index, word, intent, mark = parts
            mark = mark.strip()

            # Проверяем метку слова
            if mark.startswith('O'):
                # Если встретили 'O' и у нас есть текущий слот, завершаем его
                if current_slot_start is not None and current_slot_end is not None:
                    slots.append(f'{current_slot_start}:{current_slot_end}:{current_mark}')
                    current_slot_start = None
                    current_slot_end = None
                    current_mark = None
            else:
                # Извлекаем тип сущности из метки (удаляем префикс 'B-' или 'I-')
                entity_type = mark[2:]

                # Найдем начальный индекс слова в тексте TAT
                word_index = tat_text.find(word)

                if mark.startswith('B-'):
                    # Если слово начинается с 'B-', начинаем новый слот
                    if current_slot_start is not None and current_slot_end is not None:
                        slots.append(f'{current_slot_start}:{current_slot_end}:{current_mark}')
                    current_slot_start = word_index + 1  # Увеличиваем индекс на 1 для начала
                    current_slot_end = word_index + len(word)
                    current_mark = entity_type
                elif mark.startswith('I-') and current_mark == entity_type:
                    # Если слово начинается с 'I-' и совпадает с текущей меткой, продолжаем текущий слот
                    current_slot_end = word_index + len(word)

        # Если у нас есть незавершенный слот после прохождения абзаца, добавляем его
        if current_slot_start is not None and current_slot_end is not None:
            slots.append(f'{current_slot_start}:{current_slot_end}:{current_mark}')

        # Формируем строку для поля # slots
        slots_str = ', '.join(slots)

        # Проверяем, есть ли уже строка # slots в абзаце
        found_slots_line = False
        updated_lines = []
        for line in lines:
            if line.startswith('# slots:'):
                # Если нашли строку # slots:, заменяем её на новую строку slots_str
                updated_lines.append(f'# slots: {slots_str}')
                found_slots_line = True
            else:
                updated_lines.append(line)

        # Если строки # slots не было, добавляем её в конец абзаца
        if not found_slots_line:
            updated_lines.append(f'# slots: {slots_str}')

        # Добавляем обновленный абзац в список
        updated_paragraphs.append('\n'.join(updated_lines))

    # Переходим в начало файла и записываем обновленные абзацы
    f.seek(0)
    f.write('\n\n'.join(updated_paragraphs))
    f.truncate()  # Удаляем остаток содержимого файла, если оно есть

In [13]:
read = "udsu usdybius uisoyay intent: wefh uh ius uh zdjhi. intent: oiuea;n fdsf inte xfdxf "
intent_name = "intent: "
str_
for i in range(len(read)):
    ch = read[i:i + 7]
    if ch == intent_name:
        print(ch)

In [12]:
def process_file(input_file, output_file):
    # Функция для обработки входного файла и сохранения результата в выходной файл
    with open(input_file, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    with open(output_file, 'w', encoding='utf-8') as out_file:
        # Временные переменные для хранения данных о примере
        example_id = None
        annotator = None
        text_en = None
        text_tat = None
        intent = None
        slots = None
        token_lines = []
        
        # Обработка строк входного файла
        for line in lines:
            line = line.strip()
            
            # Обработка строк метаданных
            if line.startswith('# id:'):
                # Сохранение текущего примера перед переходом к следующему
                if example_id:
                    process_example(example_id, annotator, text_en, text_tat, intent, token_lines, out_file)
                    # Сброс переменных для следующего примера
                    example_id, annotator, text_en, text_tat, intent, token_lines = None, None, None, None, None, []
                
                # Сохранение id текущего примера
                example_id = line.split(': ')[1]
            elif line.startswith('# annotator:'):
                annotator = line.split(': ')[1]
            elif line.startswith('# text EN:'):
                text_en = line.split(': ')[1]
            elif line.startswith('# text TAT:'):
                text_tat = line.split(': ')[1]
            elif line.startswith('# intent:'):
                intent = line.split(': ')[1]
            elif line.startswith('# slots:'):
                slots = line
            else:
                # Сохранение строки токенов
                token_lines.append(line)
        
        # Обработка последнего примера
        if example_id:
            process_example(example_id, annotator, text_en, text_tat, intent, token_lines, out_file)

def process_example(example_id, annotator, text_en, text_tat, intent, token_lines, out_file):
    # Функция для обработки одного примера и записи его в выходной файл
    # Начальная и конечная позиции для каждого типа слота
    slot_ranges = {}
    
    for line in token_lines:
        # Разбор строки токена
        parts = line.split()
        if len(parts) < 3:
            continue
        
        # Извлечение позиций и слота
        position = int(parts[0])  # Позиция токена (1-индексация)
        slot = parts[2]  # Слот токена
        
        # Игнорирование слота "O"
        if slot == 'O':
            continue
        
        # Проверка префиксов слота ("B-" или "I-")
        if slot.startswith('B-'):
            # Начало новой последовательности слота
            slot_type = slot[2:]  # Тип слота
            slot_ranges[slot_type] = (position, position)
        elif slot.startswith('I-'):
            # Продолжение существующей последовательности слота
            slot_type = slot[2:]
            if slot_type in slot_ranges:
                start, end = slot_ranges[slot_type]
                slot_ranges[slot_type] = (start, position)
    
    # Формирование строки # slots:
    slots_str = ''
    for slot_type, (start, end) in slot_ranges.items():
        slots_str += f'{start}:{end}:{slot_type} '
    
    # Удаление последнего пробела в конце строки
    slots_str = slots_str.strip()
    
    # Запись обновленной строки # slots: в выходной файл
    out_file.write(f'# id: {example_id}\n')
    out_file.write(f'# annotator: {annotator}\n')
    out_file.write(f'# text EN: {text_en}\n')
    out_file.write(f'# text TAT: {text_tat}\n')
    out_file.write(f'# intent: {intent}\n')
    out_file.write(f'# slots: {slots_str}\n')
    
    # Запись строк токенов в выходной файл
    for line in token_lines:
        out_file.write(line + '\n')
    
    # Добавление пустой строки для разделения примеров
    out_file.write('\n')

# Вызов функции для обработки файла
input_file = 'en.valid_id_ann_alsu.conll'  # Укажите путь к входному файлу
output_file = 'en.valid_id_ann_alsu_updated.conll'  # Укажите путь к выходному файлу
process_file(input_file, output_file)


In [13]:
def process_file(input_file, output_file):
    # Открываем файлы для чтения и записи
    with open(input_file, 'r', encoding='utf-8') as file, open(output_file, 'w', encoding='utf-8') as out_file:
        # Временные переменные для хранения информации о текущем примере
        example_id = None
        annotator = None
        text_en = None
        text_tat = None
        intent = None
        token_lines = []
        
        # Обработка строк входного файла
        for line in file:
            line = line.strip()
            
            # Если это строка метаданных, обновляем соответствующие переменные
            if line.startswith('# id:'):
                # Сохраняем текущий пример перед переходом к следующему
                if example_id:
                    process_example(example_id, annotator, text_en, text_tat, intent, token_lines, out_file)
                    # Сброс переменных для следующего примера
                    example_id, annotator, text_en, text_tat, intent, token_lines = None, None, None, None, None, []
                
                # Сохраняем id текущего примера
                example_id = line.split(': ')[1]
                out_file.write(line + '\n')
            elif line.startswith('# annotator:'):
                annotator = line.split(': ')[1]
                out_file.write(line + '\n')
            elif line.startswith('# text EN:'):
                text_en = line.split(': ')[1]
                out_file.write(line + '\n')
            elif line.startswith('# text TAT:'):
                text_tat = line.split(': ')[1]
                out_file.write(line + '\n')
            elif line.startswith('# intent:'):
                intent = line.split(': ')[1]
                out_file.write(line + '\n')
            elif line.startswith('# slots:'):
                # Пропускаем эту строку, чтобы перезаписать ее позже после обработки токенов
                continue
            elif line.startswith('#'):
                # Пишем строку метаданных в выходной файл
                out_file.write(line + '\n')
            else:
                # Добавляем строку токенов в список
                token_lines.append(line)
        
        # Обработка последнего примера, если он существует
        if example_id:
            process_example(example_id, annotator, text_en, text_tat, intent, token_lines, out_file)

def process_example(example_id, annotator, text_en, text_tat, intent, token_lines, out_file):
    # Функция для обработки одного примера и записи его в выходной файл
    # Словарь для хранения диапазонов слотов
    slot_ranges = {}
    
    # Обработка каждой строки токенов
    for line in token_lines:
        parts = line.split()
        # Проверяем, что строка содержит информацию о токене
        if len(parts) >= 3:
            position = int(parts[0])  # Позиция токена
            slot = parts[2]  # Тип слота
            
            # Если слот не "O", обновляем словарь slot_ranges
            if slot != 'O':
                slot_type = slot
                if slot_type not in slot_ranges:
                    slot_ranges[slot_type] = [position, position]
                else:
                    slot_ranges[slot_type][1] = position
    
    # Создаем строку # slots: на основе диапазонов слотов
    slots_list = []
    for slot_type, (start, end) in slot_ranges.items():
        slots_list.append(f'{start}:{end}:{slot_type}')
    
    # Записываем строку # slots: в выходной файл
    slots_str = f'# slots: {" ".join(slots_list)}'
    out_file.write(slots_str + '\n')
    
    # Записываем строки токенов в выходной файл
    for line in token_lines:
        out_file.write(line + '\n')
    
    # Добавляем пустую строку для разделения примеров
    out_file.write('\n')

# Задайте пути к входному и выходному файлам
input_file = 'en.valid_id_ann_alsu.conll'
output_file = 'en.valid_id_ann_alsu_updated.conll'

# Обработка входного файла
process_file(input_file, output_file)


In [14]:
def process_file(input_file, output_file):
    # Открываем входной и выходной файлы
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        # Инициализируем переменные для хранения текущего примера
        example_id = None
        annotator = None
        text_en = None
        text_tat = None
        intent = None
        slots_range = []
        current_token_position = 1  # Начальная позиция текущего токена, начиная с 1
        token_lines = []

        # Обрабатываем входной файл по строкам
        for line in infile:
            # Пропускаем пустые строки
            if line.strip() == '':
                if example_id:
                    # Обрабатываем последний пример перед переходом к следующему
                    process_example(example_id, annotator, text_en, text_tat, intent, slots_range, token_lines, outfile)
                    # Сброс переменных для следующего примера
                    example_id, annotator, text_en, text_tat, intent = None, None, None, None, None
                    slots_range = []
                    current_token_position = 1
                    token_lines = []
                continue

            # Если это строка комментария, обрабатываем метаданные
            if line.startswith('#'):
                if line.startswith('# id:'):
                    example_id = line.split(': ')[1].strip()
                elif line.startswith('# annotator:'):
                    annotator = line.split(': ')[1].strip()
                elif line.startswith('# text EN:'):
                    text_en = line.split(': ')[1].strip()
                elif line.startswith('# text TAT:'):
                    text_tat = line.split(': ')[1].strip()
                elif line.startswith('# intent:'):
                    intent = line.split(': ')[1].strip()
                # Записываем строку в выходной файл
                outfile.write(line)
            else:
                # Обрабатываем токены и слоты
                token_parts = line.strip().split()
                token = token_parts[1]  # Токен
                slot = token_parts[2]  # Слот
                if slot != "O":
                    # Если слот отличается от "O", записываем его в диапазон
                    slots_range.append((current_token_position, slot))
                # Обновляем текущую позицию токена, включая пробелы
                current_token_position += len(token) + 1
                # Добавляем строку токенов в список
                token_lines.append(line)
        
        # Обрабатываем последний пример, если он существует
        if example_id:
            process_example(example_id, annotator, text_en, text_tat, intent, slots_range, token_lines, outfile)

def process_example(example_id, annotator, text_en, text_tat, intent, slots_range, token_lines, outfile):
    # Генерируем строку # slots:
    if slots_range:
        start_position = slots_range[0][0]
        end_position = slots_range[-1][0]
        slot_type = slots_range[0][1]
        slots_line = f"# slots: {start_position}:{end_position}:{slot_type}"
    else:
        slots_line = "# slots:"

    # Записываем строку # slots: в выходной файл
    outfile.write(slots_line + '\n')

    # Записываем строки токенов в выходной файл
    for line in token_lines:
        outfile.write(line)
    outfile.write('\n')

# Укажите путь к входному файлу
input_file = 'en.valid_id_ann_alsu.conll'

# Укажите путь к выходному файлу
output_file = 'en.valid_id_ann_alsu_updated.conll'

# Выполняем обработку входного файла и запись выходного файла
process_file(input_file, output_file)


In [25]:
def process_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        example_id, annotator, text_en, text_tat, intent = None, None, None, None, None
        slots = []
        current_token_index = 1
        token_lines = []
        start_position = 0
        tat_text = None

        for line in infile:
            line = line.strip()
            # Пропуск пустых строк
            if not line:
                if example_id:
                    process_example(example_id, annotator, text_en, text_tat, intent, slots, token_lines, outfile)
                    example_id, annotator, text_en, text_tat, intent = None, None, None, None, None
                    slots = []
                    current_token_index = 1
                    token_lines = []
                    start_position = 0
                continue

            # Обработка комментариев
            if line.startswith('#'):
                # Различные данные о примере
                if line.startswith('# id:'):
                    example_id = line.split(': ')[1].strip()
                elif line.startswith('# annotator:'):
                    annotator = line.split(': ')[1].strip()
                elif line.startswith('# text EN:'):
                    text_en = line.split(': ')[1].strip()
                elif line.startswith('# text TAT:'):
                    text_tat = line.split(': ')[1].strip()
                    tat_text = text_tat
                elif line.startswith('# intent:'):
                    intent = line.split(': ')[1].strip()
                outfile.write(line + '\n')
            else:
                # Обработка строк токенов
                parts = line.split('\t')
                
                # Проверка наличия достаточного количества частей
                if len(parts) < 3:
                    continue  # пропустить строку, если она не содержит достаточного количества частей
                
                token = parts[1]
                slot = parts[2]
                
                # Проверка на наличие слота, отличного от "O"
                if slot != "O":
                    token_start = start_position
                    token_end = start_position + len(token)
                    slots.append((token_start, token_end, slot))
                    
                # Обновляем начальную позицию для следующего токена
                start_position += len(token) + 1  # +1 для учета пробела
                
                # Добавляем текущую строку к списку строк токенов
                token_lines.append(line + '\n')

        # Обработка последнего примера, если он существует
        if example_id:
            process_example(example_id, annotator, text_en, text_tat, intent, slots, token_lines, outfile)

def process_example(example_id, annotator, text_en, text_tat, intent, slots, token_lines, outfile):
    # Генерация строки # slots: с учетом диапазонов позиций для токенов со слотами, отличными от "O"
    slots_line = '# slots: '
    if slots:
        slots_str = ' '.join([f'{s[0]}:{s[1]}:{s[2]}' for s in slots])
        slots_line += slots_str
    else:
        slots_line = '# slots:'

    # Запись метаданных и строки # slots: в выходной файл
    outfile.write(slots_line + '\n')
    
    # Запись строк токенов в выходной файл
    for line in token_lines:
        outfile.write(line)

# Путь к входному файлу
input_file = 'en.valid_id_ann_alsu.conll'

# Путь к выходному файлу
output_file = 'en.valid_id_ann_alsu_updated.conll'

# Запуск обработки файла
process_file(input_file, output_file)


In [17]:
def process_slots(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    current_slots = []
    current_id = None
    output_lines = []
    
    for line in lines:
        line = line.strip()
        
        # Check if the line is an ID line
        if line.startswith("# id:"):
            if current_id is not None and current_slots:
                # Output the slots information before starting a new ID block
                output_line = f"# slots: {current_slots[0]}:{current_slots[1]}:{current_slots[2]}"
                output_lines.append(output_line)
                current_slots = []
                
            # Start processing the new ID block
            current_id = line
        
        # Check if the line is a slots line
        elif line.startswith("slots:"):
            # Skip this line since we are processing tokens and will recreate slots
            continue
        
        # Check if the line contains token information
        elif line and line[0].isdigit():
            token_parts = line.split()
            # Calculate the start and end index of the token, including spaces
            start_index = len(' '.join(token_parts[:0])) + sum(len(part) + 1 for part in token_parts[:int(token_parts[0]) - 1])
            end_index = start_index + len(token_parts[1])
            
            classification = token_parts[2]
            
            # Check if the classification is not "O"
            if classification != "O":
                current_slots = [start_index, end_index, classification]
        
        # Otherwise, just add the line to the output
        else:
            output_lines.append(line)
    
    # Output the final slots information after the last ID block
    if current_id is not None and current_slots:
        output_line = f"# slots: {current_slots[0]}:{current_slots[1]}:{current_slots[2]}"
        output_lines.append(output_line)
    
    # Output the final result
    for output_line in output_lines:
        print(output_line)

# Пример использования функции
process_slots("en.valid_id_ann_alsu.conll")


# annotator: Alsu
# text EN: What are my reminders?
# text TAT: Минем нинди искә төшерүләрем бар?
# intent: reminder/show_reminders
# slots:

# slots: 30:31:reminder/show_reminders
# annotator: Alsu
# text EN: Cancel my reminder to text Dad
# text TAT: Минем әтиге язу турында хәтерләтүемне бетер
# intent: reminder/cancel_reminder
# slots:

# slots: 35:40:reminder/cancel_reminder
# annotator: Alsu
# text EN: set alarm for 6 am
# text TAT: Будильникны иртәнге 6-га куй
# intent: alarm/set_alarm
# slots:

# slots: 22:25:alarm/set_alarm
# annotator: Alsu
# text EN: turn off the alarm
# text TAT: будильникны сүндер
# intent: alarm/cancel_alarm
# slots:

# slots: 2:8:alarm/cancel_alarm
# annotator: Alsu
# text EN: cancel the alarm
# text TAT: будильникны бетер
# intent: alarm/cancel_alarm
# slots:

# slots: 2:7:alarm/cancel_alarm
# annotator: Alsu
# text EN: turn off alarm
# text TAT: будильникны сүндер
# intent: alarm/cancel_alarm
# slots:

# slots: 2:8:alarm/cancel_alarm
# annotator: Alsu
#

In [18]:
def process_slots(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    # Параметры для текущей записи
    current_id = None
    slots_info = []

    for line in lines:
        # Проверяем на заголовок ID
        if line.startswith("# id:"):
            if current_id:
                # Выводим информацию о слотах для текущей записи перед переходом к следующей
                if slots_info:
                    print(f"# slots: {':'.join(slots_info)}")
                
                # Сброс текущих данных
                slots_info = []
                
            # Устанавливаем текущий ID
            current_id = line.strip()
        
        # Проверяем на слот или токен
        elif line and line[0].isdigit():
            parts = line.split()
            token_idx = int(parts[0])  # Индекс токена
            token = parts[1]  # Токен
            classification = parts[2]  # Классификация

            # Если классификация не "O", рассчитываем индексы
            if classification != "O":
                start_index = sum(len(part) + 1 for part in parts[0:1]) + len(parts[1]) * (token_idx - 1)
                end_index = start_index + len(token) - 1
                slots_info.append(f"{start_index}:{end_index}:{classification}")
        
        # Обрабатываем строку
        else:
            if line.strip().startswith("# slots:"):
                continue
            
            print(line.strip())
    
    # Вывод последней информации о слотах для текущей записи
    if slots_info:
        print(f"# slots: {':'.join(slots_info)}")

# Используйте функцию с вашим файлом
process_slots("en.valid_id_ann_alsu.conll")


# annotator: Alsu
# text EN: What are my reminders?
# text TAT: Минем нинди искә төшерүләрем бар?
# intent: reminder/show_reminders

# slots: 2:6:reminder/show_reminders:7:11:reminder/show_reminders:10:13:reminder/show_reminders:35:45:reminder/show_reminders:14:16:reminder/show_reminders:7:7:reminder/show_reminders
# annotator: Alsu
# text EN: Cancel my reminder to text Dad
# text TAT: Минем әтиге язу турында хәтерләтүемне бетер
# intent: reminder/cancel_reminder

# slots: 2:6:reminder/cancel_reminder:7:11:reminder/cancel_reminder:8:10:reminder/cancel_reminder:23:29:reminder/cancel_reminder:54:66:reminder/cancel_reminder:27:31:reminder/cancel_reminder
# annotator: Alsu
# text EN: set alarm for 6 am
# text TAT: Будильникны иртәнге 6-га куй
# intent: alarm/set_alarm

# slots: 2:12:alarm/set_alarm:9:15:alarm/set_alarm:10:13:alarm/set_alarm:11:13:alarm/set_alarm
# annotator: Alsu
# text EN: turn off the alarm
# text TAT: будильникны сүндер
# intent: alarm/cancel_alarm

# slots: 2:12:alarm/

In [19]:
def process_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    # Параметры для текущей записи
    current_id = None
    current_slots = []
    current_text = ""
    inside_slots_section = False

    for line in lines:
        if line.startswith("# id:"):
            # Если есть текущий id и накопленные слоты, то выводим информацию
            if current_id and current_slots:
                # Вывод строки # slots в правильном формате
                slots_str = f"# slots: {' '.join(current_slots)}"
                print(slots_str)

            # Начинаем новую запись
            current_id = line.strip()
            current_slots = []
            current_text = ""
            inside_slots_section = False
        
        # Проверяем начало секции slots
        elif line.strip() == "# slots:":
            inside_slots_section = True
        
        # Обрабатываем строку в секции slots
        elif inside_slots_section:
            if line.startswith("#"):
                # Выход из секции slots
                inside_slots_section = False
            elif line.strip():
                parts = line.split()
                token_idx = int(parts[0])  # Индекс токена (1-based)
                token = parts[1]  # Токен
                classification = parts[2]  # Классификация
                
                # Если классификация не "O", рассчитываем индексы
                if classification != "O":
                    # Рассчитываем start_index и end_index
                    if current_text == "":
                        start_index = 0
                    else:
                        start_index = len(current_text) + 1  # Плюс 1 пробел

                    end_index = start_index + len(token) - 1
                    
                    # Добавляем текущий токен к текущему тексту
                    current_text += " " + token if current_text else token
                    
                    # Формируем строку для текущего слота
                    slot_str = f"{start_index}:{end_index}:{classification}"
                    current_slots.append(slot_str)
            else:
                # Пустая строка означает конец текущей записи
                inside_slots_section = False
        
        # Просто печатаем строку
        print(line.strip())
    
    # Вывод последней информации о слотах для текущей записи
    if current_id and current_slots:
        slots_str = f"# slots: {' '.join(current_slots)}"
        print(slots_str)

# Используйте функцию с вашим файлом
process_file("en.valid_id_ann_alsu.conll")


# id: valid_101
# annotator: Alsu
# text EN: What are my reminders?
# text TAT: Минем нинди искә төшерүләрем бар?
# intent: reminder/show_reminders
# slots:
1	Минем	reminder/show_reminders	B-reference
2	нинди	reminder/show_reminders	O
3	искә	reminder/show_reminders	O
4	төшерүләрем	reminder/show_reminders	O
5	бар	reminder/show_reminders	O
6	?	reminder/show_reminders	O

# slots: 0:4:reminder/show_reminders 6:10:reminder/show_reminders 12:15:reminder/show_reminders 17:27:reminder/show_reminders 29:31:reminder/show_reminders 33:33:reminder/show_reminders
# id: valid_102
# annotator: Alsu
# text EN: Cancel my reminder to text Dad
# text TAT: Минем әтиге язу турында хәтерләтүемне бетер
# intent: reminder/cancel_reminder
# slots:
1	Минем	reminder/cancel_reminder	B-reference
2	әтиге	reminder/cancel_reminder	I-reminder/todo
3	язу	reminder/cancel_reminder	B-reminder/todo
4	турында	reminder/cancel_reminder	O
5	хәтерләтүемне	reminder/cancel_reminder	O
6	бетер	reminder/cancel_reminder	O

# slots: 0

In [20]:
def process_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    # Параметры для текущей записи
    current_id = None
    current_slots = []
    inside_slots_section = False

    for line in lines:
        # Если найдена строка # id:
        if line.startswith("# id:"):
            # Если текущая запись уже заполнена, выводим результат
            if current_slots:
                print(f"# slots: {' '.join(current_slots)}")
            
            # Инициализируем новую запись
            current_id = line.strip()
            current_slots = []
            inside_slots_section = False
        
        # Если найдено начало секции # slots:
        elif line.strip() == "# slots:":
            inside_slots_section = True
            continue
        
        # Если внутри секции # slots:, обрабатываем строки
        elif inside_slots_section:
            if line.startswith("#"):
                # Если найден комментарий (новая секция), значит конец текущей секции
                inside_slots_section = False
            elif line.strip():
                # Парсим строку с токенами
                parts = line.split()
                token_idx = int(parts[0])  # Индекс токена (1-based)
                classification = parts[2]  # Классификация
                
                # Если классификация не "O", добавляем слот к текущему списку слотов
                if classification != "O":
                    # Рассчитываем start_index и end_index
                    start_index = line.find(parts[1])
                    end_index = start_index + len(parts[1]) - 1
                    
                    # Создаем строку для слота и добавляем ее в список
                    slot_str = f"{start_index}:{end_index}:{classification}"
                    current_slots.append(slot_str)
            else:
                # Пустая строка означает конец текущей записи
                inside_slots_section = False
        
        # Печатаем исходную строку (исключая конец строки)
        print(line.strip())
    
    # Если есть последние слоты в текущей записи, выводим их
    if current_slots:
        print(f"# slots: {' '.join(current_slots)}")

# Используйте функцию с вашим файлом
process_file("en.valid_id_ann_alsu.conll")


# id: valid_101
# annotator: Alsu
# text EN: What are my reminders?
# text TAT: Минем нинди искә төшерүләрем бар?
# intent: reminder/show_reminders
1	Минем	reminder/show_reminders	B-reference
2	нинди	reminder/show_reminders	O
3	искә	reminder/show_reminders	O
4	төшерүләрем	reminder/show_reminders	O
5	бар	reminder/show_reminders	O
6	?	reminder/show_reminders	O

# slots: 2:6:reminder/show_reminders 2:6:reminder/show_reminders 2:5:reminder/show_reminders 2:12:reminder/show_reminders 2:4:reminder/show_reminders 2:2:reminder/show_reminders
# id: valid_102
# annotator: Alsu
# text EN: Cancel my reminder to text Dad
# text TAT: Минем әтиге язу турында хәтерләтүемне бетер
# intent: reminder/cancel_reminder
1	Минем	reminder/cancel_reminder	B-reference
2	әтиге	reminder/cancel_reminder	I-reminder/todo
3	язу	reminder/cancel_reminder	B-reminder/todo
4	турында	reminder/cancel_reminder	O
5	хәтерләтүемне	reminder/cancel_reminder	O
6	бетер	reminder/cancel_reminder	O

# slots: 2:6:reminder/cancel_reminde

In [24]:
def process_slots(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Параметры для текущей записи
    current_id = None
    current_slots_output = []
    inside_slots_section = False
    slot_strs = []

    for line in lines:
        # Если найдена строка # id:
        if line.startswith("# id:"):
            # Если текущая запись уже заполнена, выводим результат
            if current_id and slot_strs:
                # Формируем строку слотов
                formatted_slots = " : ".join(slot_strs)
                print(f"# slots: {formatted_slots}")
                slot_strs = []
            
            # Инициализируем новую запись
            current_id = line.strip()
            inside_slots_section = False

        # Если найдено начало секции # slots:
        elif line.strip() == "# slots:":
            inside_slots_section = True
            continue
        
        # Если внутри секции # slots:, обрабатываем строки
        elif inside_slots_section:
            if line.startswith("#") or not line.strip():
                # Конец секции слотов
                inside_slots_section = False
            else:
                # Разбираем строку токена
                parts = line.split()
                token_index = int(parts[0])  # Индекс токена
                classification = parts[2]  # Классификация (intent)
                token_tag = parts[3]  # Тег (e.g. B-reference)

                # Если классификация не O, добавляем в список слотов
                if classification != "O":
                    slot_strs.append(f"{token_index}:{token_index}:{token_tag}")

        # Выводим оригинальные строки (с исключением строк слотов)
        if not line.strip().startswith("# slots:") and not inside_slots_section:
            print(line.strip())

    # Если есть последние слоты в текущей записи, выводим их
    if current_id and slot_strs:
        formatted_slots = " : ".join(slot_strs)
        print(f"# slots: {formatted_slots}")

# Используйте функцию с вашим файлом
process_slots("en.valid_id_ann_alsu.conll")


# id: valid_101
# annotator: Alsu
# text EN: What are my reminders?
# text TAT: Минем нинди искә төшерүләрем бар?
# intent: reminder/show_reminders

# slots: 1:1:B-reference : 2:2:O : 3:3:O : 4:4:O : 5:5:O : 6:6:O
# id: valid_102
# annotator: Alsu
# text EN: Cancel my reminder to text Dad
# text TAT: Минем әтиге язу турында хәтерләтүемне бетер
# intent: reminder/cancel_reminder

# slots: 1:1:B-reference : 2:2:I-reminder/todo : 3:3:B-reminder/todo : 4:4:O : 5:5:O : 6:6:O
# id: valid_103
# annotator: Alsu
# text EN: set alarm for 6 am
# text TAT: Будильникны иртәнге 6-га куй
# intent: alarm/set_alarm

# slots: 1:1:O : 2:2:B-datetime : 3:3:I-datetime : 4:4:O
# id: valid_104
# annotator: Alsu
# text EN: turn off the alarm
# text TAT: будильникны сүндер
# intent: alarm/cancel_alarm

# slots: 1:1:O : 2:2:O
# id: valid_105
# annotator: Alsu
# text EN: cancel the alarm
# text TAT: будильникны бетер
# intent: alarm/cancel_alarm

# slots: 1:1:O : 2:2:O
# id: valid_106
# annotator: Alsu
# text EN